In [18]:
import silence_tensorflow.auto
import tensorflow as tf
gpuDevices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpuDevices))
tf.config.experimental.set_memory_growth(gpuDevices[0], True)
tf.config.optimizer.set_experimental_options({'layout_optimizer': False})

import keras
import random
import PIL.Image
import glob
import torchshow
import transformers
import random
from  IPython import display
import time
import datasets
import matplotlib.pyplot as plt
import numpy as np


Num GPUs Available:  1


In [12]:
ts = lambda tensor : keras.preprocessing.image.array_to_img(tensor)

def plot_n_samples(dataset, n ):
    plt.figure(figsize=(20, 4))
    for i in range(n):
        assert tf.is_tensor(dataset[i]), "This ain't a tensor"
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(ts(dataset[i].numpy()))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [8]:
import gc
gc.collect()

8640

In [7]:
ds = datasets.load_dataset("project-sloth/captcha-images", split='train')

In [24]:
def perf(method, *args, **kwargs):
    start_time = time.perf_counter()
    result = method(*args, **kwargs)
    end_time = time.perf_counter()
    return end_time - start_time

In [29]:
def lambda_conversion(ds: datasets.Dataset):
    return ds.map(lambda x: {"image": x["image"].convert("L")})


In [33]:
def convert_to_grayscale(image):
    return tf.image.rgb_to_grayscale(image)

def tf_conversion(ds: datasets.Dataset):
    ds = ds.with_format("tf")
    def map_fn(x):
        return {"image": convert_to_grayscale(x['image'])}
    return ds.map(map_fn)


In [31]:
time_pillow = perf(lambda_conversion,ds)
print(f"Time taken with Pillow: {time_pillow} seconds")

Time taken with Pillow: 0.004767084999912186 seconds


In [34]:
time_tf = perf(tf_conversion,ds)
print(f"Time taken with TensorFlow: {time_tf} seconds")

Map:  16%|█▋        | 985/6000 [00:21<00:12, 401.80 examples/s]/home/navid/.pyenv/versions/3.11.9/envs/tf/lib/python3.11/site-packages/datasets/features/image.py:339: UserWarning: Downcasting array dtype int64 to uint8 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
Map:  17%|█▋        | 999/6000 [00:49<25:59,  3.21 examples/s] 